In [1]:
author = 'Will Misenheimer'
title = 'Fake News Classifier'
date = '4/14/2020'

# Purpose #
## The purpose of this document is to build a model (Bag of words + Naive Bayes) to detect fake news and understand what separates fake from legitimate news

In [2]:
## Import packages
import pandas as pd
import numpy as np
import os # to change working directory
import nltk # nltk package
import re # regular expressions
from nltk.corpus import stopwords # stopwords
from gensim.corpora.dictionary import Dictionary # for topic modeling
from nltk.tokenize import word_tokenize # to tokenize
from gensim.models.tfidfmodel import TfidfModel # to create Tfidf model with gensim
import spacy # for named entity recognition
import en_core_web_sm # english spacy model
from sklearn.model_selection import train_test_split # to split data into train and test sets
from sklearn.feature_extraction.text import CountVectorizer # Converts text documents to matrix of token counts
from sklearn.naive_bayes import MultinomialNB # Naive Bayes Classifer
from sklearn import metrics # to measure model performance

## Read in data

In [3]:
## set working directory
os.chdir("C:/Users/willm/Documents/Projects/Fake News Classifier/fake-and-real-news-dataset")

## read in legitimate and fake news
real = pd.read_csv("True.csv")
fake = pd.read_csv("Fake.csv")

In [4]:
## Add "class" field for fake news and real news
real['class'] = "real"
fake['class'] = "fake"

In [5]:
## View datasets
real.head(10)

,title,text,subject,date,class
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",real
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",real
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",real
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",real
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",real
5,"White House, Congress prepare for talks on spe...","WEST PALM BEACH, Fla./WASHINGTON (Reuters) - T...",politicsNews,"December 29, 2017",real
6,"Trump says Russia probe will be fair, but time...","WEST PALM BEACH, Fla (Reuters) - President Don...",politicsNews,"December 29, 2017",real
7,Factbox: Trump on Twitter (Dec 29) - Approval ...,The following statements were posted to the ve...,politicsNews,"December 29, 2017",real
8,Trump on Twitter (Dec 28) - Global Warming,The following statements were posted to the ve...,politicsNews,"December 29, 2017",real
9,Alabama official to certify Senator-elect Jone...,WASHINGTON (Reuters) - Alabama Secretary of St...,politicsNews,"December 28, 2017",real


In [6]:
fake.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",fake
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",fake
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",fake
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",fake
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",fake
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",fake
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",fake
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",fake
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",fake
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",fake


In [7]:
## concatenate fake and real news dataframes
news = pd.concat([fake, real], ignore_index=True)

In [8]:
news.head()

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",fake
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",fake
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",fake
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",fake
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",fake


## Tokenize the text of each document

In [9]:
## Use list comprehension to tokenize the text of each document
tokenized_text = [word_tokenize(text.lower())
                 for text in news['text']]

In [10]:
## Create a gensim Dictionary -- assigns an ID to each unique word
dictionary = Dictionary(tokenized_text) 

In [11]:
#dictionary.token2id

## Create a gensim corpus

In [12]:
corpus = [dictionary.doc2bow(doc) for doc in tokenized_text]

In [13]:
#corpus

## TF-IDF with gensim

In [14]:
tfidf = TfidfModel(corpus)

In [15]:
#tfidf[corpus[1]]

# Named Entity Recognition

### Used to identify important entities in our documents. These can be:
    ### People
    ### Places
    ### Things
    ### Dates

In [16]:
## Initialize spacy model using English language
nlp = spacy.load('en_core_web_sm')

In [17]:
nlp.entity

In [18]:
## Print the first document
news['text'][1]

'House Intelligence Committee Chairman Devin Nunes is going to have a bad day. He s been under the assumption, like many of us, that the Christopher Steele-dossier was what prompted the Russia investigation so he s been lashing out at the Department of Justice and the FBI in order to protect Trump. As it happens, the dossier is not what started the investigation, according to documents obtained by the New York Times.Former Trump campaign adviser George Papadopoulos was drunk in a wine bar when he revealed knowledge of Russian opposition research on Hillary Clinton.On top of that, Papadopoulos wasn t just a covfefe boy for Trump, as his administration has alleged. He had a much larger role, but none so damning as being a drunken fool in a wine bar. Coffee boys  don t help to arrange a New York meeting between Trump and President Abdel Fattah el-Sisi of Egypt two months before the election. It was known before that the former aide set up meetings with world leaders for Trump, but team Tr

In [19]:
## Print the entities of the first document
doc = nlp(news['text'][1]) # example of Named Entity Recognition with the first article
doc.ents

(House Intelligence Committee,
 Devin Nunes,
 a bad day,
 Christopher Steele-dossier,
 Russia,
 the Department of Justice,
 FBI,
 Trump,
 the New York Times,
 Trump,
 George Papadopoulos,
 Russian,
 Hillary Clinton,
 Papadopoulos,
 Trump,
 New York,
 Trump,
 Abdel Fattah el-Sisi,
 Egypt,
 two months,
 Trump,
 Trump,
 May 2016,
 Papadopoulos,
 Australian,
 Alexander Downer,
 Russian,
 Democratic,
 Hillary Clinton,
 Papadopoulos,
 night,
 the Kensington Wine Rooms,
 Australian,
 Alexander Downer,
 two months later,
 Democratic,
 Australian,
 Papadopoulos,
 American,
 four,
 American,
 Australians,
 Papadopoulos,
 F.B.I.,
 Robert Mueller,
 Win McNamee/)

### Get most popular Entites for all of text

In [20]:
#doc_entities = [nlp(text.lower()) for text in news['text']]

## Build Classification Model to classify news as fake or not fake

In [21]:
## Define response variable
y = news['class'] # get y-variable

In [22]:
## Randomly split the data into train (70%) and test (30%)
X_train, X_test, y_train, y_test = train_test_split(news['text'], y, test_size=0.3, random_state=69) # split data

In [23]:
## convert text of documents to matrix of tokens
count_vectorizer = CountVectorizer(stop_words='english') # bag of words

In [24]:
## Convert training data to input format for scikit-learn and fit the model
count_train = count_vectorizer.fit_transform(X_train.values) # fit model

In [25]:
## Convert testing data to input for scikit-learn
count_test = count_vectorizer.transform(X_test.values)

In [26]:
## Create instance of Naive Bayes classifier
nb_classifier = MultinomialNB()

In [27]:
## Fit training data to Naive Bayes classifier
nb_classifier.fit(count_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [28]:
## Predict the class of each of the documents in the testing data using the model built on training data
pred = nb_classifier.predict(count_test)

In [29]:
## Accuracy of the model
metrics.accuracy_score(y_test, pred) # accuracy

0.9485523385300668

In [30]:
## Confusion matrix showing correct and incorrect classifications
metrics.confusion_matrix(y_test, pred, labels=["real", "fake"])

array([[6097,  318],
       [ 375, 6680]], dtype=int64)

## TO-DO: Investigate the articles that we misclassified

In [41]:
X_test2 = pd.DataFrame(X_test)

In [42]:
X_test2.head(10)

,text
21347,"As it turns out, police brutality really isn t..."
2583,Donald Trump s Education Secretary is off to a...
22746,Americans have lost the plot when it comes to ...
4756,Republican presidential nominee Donald Trump s...
35759,MOSCOW (Reuters) - Russian President Vladimir ...
37092,TEGUCIGALPA (Reuters) - The Honduran president...
29136,(Reuters) - U.S. Customs & Border Protection (...
30291,WASHINGTON (Reuters) - One of Donald Trump’s p...
6560,Amnesty International recently put out a brill...
39941,BEIJING (Reuters) - Chinese President Xi Jinpi...


In [43]:
X_test2['pred'] = pred

In [44]:
X_test2['class'] = y_test

In [48]:
## Misclassified articles
mis = X_test2[X_test2['pred'] != X_test2['class']]

In [52]:
mis['text'].iloc[2]

'President Obama blasted Republican presidential front-runner Donald Trump in April, 2016 for his plan to bar immigrants in the U.S. from sending money back home to force Mexico to pay for a new border wall. Oh, no. It s Trump,  a smiling Obama said when he was asked about the 2016 candidate s latest comments.Obama called plan another shortsighted idea from Trump that could spark concern among world leaders.  I am getting questions constantly from foreign leaders about some of the wackier suggestions that are being made,  Obama told reporters at the White House.Obama said Trump s border wall proposal could have  enormous  economic consequences for Mexico, which receives billions of dollars in payments that immigrants in the U.S. send home, and possibly touch off a major spat with an ally. The notion that we are going to track every Western Union bit of money that s being sent to Mexico   good luck with that,  Obama said.Obama s remarks came after The Washington Post published a two-pag

In [53]:
mis_entities = [nlp(text) for text in mis['text']]

In [68]:
type(mis_entities[11].ents)

tuple

## Topic Modeling of Misclassified Articles